In [1]:
import os
import datetime
import pandas as pd
import numpy as np
import re

In [2]:
path_raw = '//icnas4.cc.ic.ac.uk/rs5115/MSci Project/raw_spectra' #path with raw spectra files

In [3]:
file_array = []     #empty array to append filenames in original order (wrong order)
all_the_files = np.array([i for i in os.listdir(path_raw) if os.path.isfile(os.path.join(path_raw,i)) and i.startswith('IC_raw_spectra_')]) #all files in directory starting with 'IC_raw_spectra'

for f in all_the_files:
    file_array.append(f.strip('IC_raw_spectra.tx'))      #stripping file names to get the number at the end

file_integers = sorted([int(i) for i in file_array])     #number converted to sorted integers (not naturally done)


all_files_list = []    #final file list, in order

for num in file_integers: #add integer to string to make files_list
    all_files_list.append('IC_raw_spectra_'+str(num)+'.txt')

In [4]:
"""removing files without associated temperature measurement"""

def read_temperatures_check(directory, filename):
    lines = open(os.path.join(directory, filename)).readlines() #finds all lines in file as string elements in "lines" list
    temp1 = [float(b) for b in (lines[1].split())[6:]]
    tempHBB1 = temp1[0]

    return tempHBB1

In [5]:
delete_files = [] #files where there is no temperature measurement

for f in all_files_list:
    if read_temperatures_check(path_raw, f) == 0.0:
        delete_files.append(f)

files_list = [x for x in all_files_list if x not in delete_files]

In [6]:
"""retrieving times from files"""

def read_times(directory, filename):
    lines = open(os.path.join(directory, filename)).readlines() #finds all lines in file as string elements in "lines" list
    
    times = lines[0].split() #retrieve times of beginning of all spectra in a file
    return times

In [7]:
all_times_list = [] #final time list [374x4] (in HH:MM:SS.microsec format)

for f in range(len(files_list)): #go through files_list and extract times
    all_times_list.append(read_times(path_raw, files_list[f]))

flat_list = [item for sublist in all_times_list for item in sublist] #1D list of all times (in HH:MM:SS.microsec format)


spectrum_times = [] #final time list [(374*4)x1] in seconds from midnight format

for i in flat_list: #convert times to seconds format
    time = datetime.datetime.strptime(i, '%H:%M:%S.%f')
    spectrum_times.append((time.microsecond)/1000000 +time.second+time.minute*60+time.hour*3600)

def read_raw_spectra(directory, filename):
    lines = open(os.path.join(directory, filename)).readlines() #finds all lines in file as string elements in "lines" list
    
    times = lines[0].split()
    wns = []
    sp1 = []
    sp2 = []
    sp3 = []
    sp4 = []
    
    temp1 = [float(b) for b in (lines[1].split())[6:]]
    temp2 = [float(b) for b in (lines[2].split())[6:]]
    temp3 = [float(b) for b in (lines[3].split())[6:]]
    temp4 = [float(b) for b in (lines[4].split())[6:]]
    
    for i in lines[5:]:
        line = i.split()
        final = [float(a) for a in line]
        
        wns.append(final[0])
        sp1.append(final[1])
        sp2.append(final[2])
        sp3.append(final[3])
        sp4.append(final[4])

    return times #wns, sp1, sp2, sp3, sp4, times, temp1, temp2, temp3, temp4